In [30]:
from powersimdata.input.grid import Grid
from prereise.gather.winddata.rap import rap, helpers # impute

import pandas as pd
from matplotlib import pyplot
import numpy as np
import utm
import math

Wind Farms in Network + Branches/lines & buses

In [2]:
grid = Grid(["Texas"])

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


In [25]:
buses = grid.bus

In [26]:
#  X and Y coordinates of buses 
result = utm.from_latlon(buses.lat.to_numpy(),buses.lon.to_numpy())
buses['X'], buses['Y'] = result[0], result[1]

In [7]:
buses.head()

,type,Pd,Qd,Gs,Bs,zone_id,Vm,Va,baseKV,loss_zone,...,Vmin,lam_P,lam_Q,mu_Vmax,mu_Vmin,interconnect,lat,lon,X,Y
bus_id,,,,,,,,,,,,,,,,,,,,,
3001001,1,20.78,5.89,0,0.0,301,0.998372,-22.3462,115.0,9,...,0.9,0.0,0,0,0,Texas,31.9067,-102.262,758924.088679,3.533366e+06
3001002,1,15.41,4.37,0,0.0,301,1.022910,-17.8585,115.0,9,...,0.9,0.0,0,0,0,Texas,29.8880,-104.519,546443.618177,3.306472e+06
3001003,1,0.00,0.00,0,0.0,301,1.014100,-17.5032,115.0,9,...,0.9,0.0,0,0,0,Texas,32.9264,-101.648,813463.290141,3.648115e+06
3001004,2,0.00,0.00,0,0.0,301,1.020050,-19.3247,230.0,9,...,0.9,0.0,0,0,0,Texas,32.9264,-101.648,813463.290141,3.648115e+06
3001005,1,0.00,0.00,0,0.0,301,1.012380,-15.9276,115.0,9,...,0.9,0.0,0,0,0,Texas,32.2075,-101.388,840500.219088,3.569161e+06


In [27]:
branches = grid.branch
branches.head()

,from_bus_id,to_bus_id,r,x,b,rateA,rateB,rateC,ratio,angle,...,branch_device_type,interconnect,from_zone_id,to_zone_id,from_zone_name,to_zone_name,from_lat,from_lon,to_lat,to_lon
branch_id,,,,,,,,,,,,,,,,,,,,,
100915,3001001,3001064,0.00524,0.03580,0.00609,221.0,0.0,0.0,0.0,0.0,...,Line,Texas,301,301,Far West,Far West,31.9067,-102.262,32.0011,-102.312
100916,3001001,3001064,0.00524,0.03580,0.00609,221.0,0.0,0.0,0.0,0.0,...,Line,Texas,301,301,Far West,Far West,31.9067,-102.262,32.0011,-102.312
100917,3001001,3001071,0.00435,0.02808,0.00536,221.0,0.0,0.0,0.0,0.0,...,Line,Texas,301,301,Far West,Far West,31.9067,-102.262,31.8403,-102.326
100918,3001001,3001071,0.00435,0.02808,0.00536,221.0,0.0,0.0,0.0,0.0,...,Line,Texas,301,301,Far West,Far West,31.9067,-102.262,31.8403,-102.326
100919,3001002,3001007,0.07776,0.46693,0.06336,98.0,0.0,0.0,0.0,0.0,...,Line,Texas,301,301,Far West,Far West,29.8880,-104.519,31.0935,-104.625


In [9]:
branches.columns

Index(['from_bus_id', 'to_bus_id', 'r', 'x', 'b', 'rateA', 'rateB', 'rateC',
       'ratio', 'angle', 'status', 'angmin', 'angmax', 'Pf', 'Qf', 'Pt', 'Qt',
       'mu_Sf', 'mu_St', 'mu_angmin', 'mu_angmax', 'branch_device_type',
       'interconnect', 'from_zone_id', 'to_zone_id', 'from_zone_name',
       'to_zone_name', 'from_lat', 'from_lon', 'to_lat', 'to_lon'],
      dtype='object')

In [28]:
branches.shape

(3206, 31)

In [29]:
# Assign start and end X and Y coordinates
from_result = utm.from_latlon(branches.from_lat.to_numpy(),branches.from_lon.to_numpy())
branches['fromX'], branches['fromY'] = from_result[0], from_result[1]

to_result = utm.from_latlon(branches.to_lat.to_numpy(),branches.to_lon.to_numpy())
branches['toX'], branches['toY'] = to_result[0], to_result[1]


In [53]:
5//0.3

16.0

In [54]:
np.linspace(start=0,stop=5,num=16)

array([0.        , 0.33333333, 0.66666667, 1.        , 1.33333333,
       1.66666667, 2.        , 2.33333333, 2.66666667, 3.        ,
       3.33333333, 3.66666667, 4.        , 4.33333333, 4.66666667,
       5.        ])

In [59]:
np.linspace(0,1,10)[9]

1.0

In [ ]:
def sample_points(x1,y1,x2,y2,d):
    # Start: (x1,y1)
    # End: (x2,y2)
    # d: step size
    slope = (y2 - y1)/(x2 - x1)
    total_dist = math.sqrt((y2-y1)**2 + (x2-x1)**2)
    num_points = int(total_dist//d)
    points = np.zeros((num_points,2)) # x and y coordinates of sample points
    dist_points = np.linspace(start=0.0,stop=total_dist,num=num_points) # incremental distance along the line

    dx = total_dist/math.sqrt(1+slope**2)
    dy = slope*dx

    points[0,0:] = [x1,y1] # Start point
    points[num_points-1,:] = [x2,y2] # End point
    for i in range(1,num_points-1): 
        points[i,0] = points[i-1,0] + dx
        points[i,1] = points[i-1,1] + dy